# Funzioni
## Funzioni Built-in

<a id='partial'></a>
### partial

Per funzioni complesse con molti argomenti, da richiamare piu volte senza specificare sempre gli stessi argomenti

In [1]:
from functools import partial

In [2]:
def maggiore_di(a, b):
    return a > b
print(maggiore_di(15, 10))

True


In [3]:
maggiore_di_20 = partial(maggiore_di, b=20)
print(maggiore_di_20(15))

False


Partial rende costante uno o più argomenti della funzione, questo per poterla richiamare più volte in più modi ben leggibili. 
Da notare che <U>l'implementazione</U> della funzione però, <U>è sempre la stessa</U>.

Se si vuole modificare l'implementazione a seconda del tipo di argomento inserito, usare 
[@singledispatch](#singledispatch)

Vi è anche la versione per i metodi nelle classi, sempre in functools, ed è [partialmethod](#partialmethod)

Per maggiori dettagli la <a href="https://docs.python.org/3/library/functools.html">documentazione ufficiale di functools</a>.

## Decoratori di Funzioni
### Creazione di un decoratore di funzioni
Il decoratore si ciuccia la funzione, ne estrapola gli argomenti args e le keywords kwargs, se li lavora e poi gli rimette dentro i valori args e kwargs modificati.

Creiamo l'implementazione del decoratore

Per funzioni complesse con molti argomenti, da richiamare piu volte senza specificare sempre gli stessi argomenti

In [4]:
from functools import partial

In [5]:
def maggiore_di(a, b):
    return a > b
print(maggiore_di(15, 10))

True


In [6]:
maggiore_di_20 = partial(maggiore_di, b=20)
print(maggiore_di_20(15))

False


Partial rende costante uno o più argomenti della funzione, questo per poterla richiamare più volte in più modi ben leggibili. 
Da notare che <U>l'implementazione</U> della funzione però, <U>è sempre la stessa</U>.

Se si vuole modificare l'implementazione a seconda del tipo di argomento inserito, usare [@singledispatch](#@singledispatch)

Vi è anche la versione per i metodi nelle classi, sempre in functools, ed è [partialmethod](#partialmethod)

Per maggiori dettagli la <a href="https://docs.python.org/3/library/functools.html">documentazione ufficiale di functools</a>.

In [7]:
def decoratore(funzione):
    def wrapper(*args, **kwargs):
        """Documentazione del wrapper - non ci interessa"""
        print(args)
        print(kwargs)
        output = funzione(*args, **kwargs)
        return output
    return wrapper

La documentazione che vediamo qui servirà per capire il prossimo capitolo, per adesso non ci interessa. Accenniamo il problema in fondo a questo paragrafo

Applichiamo il decoratore ad una funzione dichiarata e implementata

In [8]:
@decoratore
def cavia(a, b):
    """Documentazione di cavia - questa è quella che vogliamo"""
    return a + b

Richiamo la funzione, e ci metto apposta un argomento e una keyword, giusto per vedere l'intero output

In [9]:
cavia(5,b=5)

(5,)
{'b': 5}


10

Abbiamo un piccolo problema, la documentazione non funziona bene cosi:

In [10]:
print(cavia.__name__)
print(cavia.__doc__)

wrapper
Documentazione del wrapper - non ci interessa


Da come vediamo, abbiamo richiamato il nome della funzione e la sua documentazione, ma abbiamo ottenuto quelle del wrapper. Questo non è molto pythonico, specialmente per problemi di mantenibilità del codice. Vediamo come risolvere

<a id='wraps'></a>
### @wraps
Aggiungendo il decoratore @wraps andiamo a risolvere il problema della documentazione, vediamo come.

Per prima cosa da functools importiamo il modulo wraps

In [11]:
from functools import wraps

Dopodichè possiamo aggiungere il decoratore wraps con funzione nel suo argomento

In [12]:
def decoratore(funzione):
    @wraps(funzione)
    def wrapper(*args, **kwargs):
        """Documentazione del wrapper - non ci interessa"""
        print(args)
        print(kwargs)
        output = funzione(*args, **kwargs)
        return output
    return wrapper

@decoratore
def cavia(a, b):
    """Documentazione di cavia - questa è quella che vogliamo"""
    return a + b

A questo punto se andiamo a provare ottenziamo quello che vogliamo

In [13]:
print(cavia.__name__)
print(cavia.__doc__)

cavia
Documentazione di cavia - questa è quella che vogliamo


<a id='singledispatch'></a>
### @singledispatch (Overloading funzioni)
È un decoratore di funzioni built-in compreso in functools per l'Overloading

Con questa funzione posso chiamare una funzione con diversi set di parametri, ovvero l'overloading, senza dover usare dei cicli if else per smistare i dati e fare diversi return nella funzione. Ma ATTENZIONE, <U>si chiama single dispatch proprio perchè quello che fa da dispatcher è il type del primo argomento che gli passiamo.</U> Inoltre funzionano solo sulle funzioni e non sui metodi nelle classi, per quello vedere [singledispatchmethod](#singledispatchmethod).

Per prima cosa richiamiamo il modulo che ci interessa

In [14]:
from functools import singledispatch

Poi creiamo la nostra funzione con il decoratore

In [15]:
@singledispatch
def funzione(*args, **kwargs):
    raise NotImplementedError(f"Il tipo non è supportato")

Questa prima definizione <U>viene eseguita solo se i successivi Overloading non combaciano con i dispatcher che abbiamo definito, ovvero funge da "fallback"</U>. Tuttavia è possibile scrivere codice qui dentro se serve. L'errore viene visualizzato se gli argomenti implementati (ancora non lo abbiamo fatto) non corrispondono con quelli chi implementeremo adesso

Comunque, vediamo la versione implementata della prima parte di fallback, può servire in molti casi

In [16]:
@singledispatch
def funzione(arg1, arg2):
    try:
        result = arg1 + arg2 # implementazione se va bene il try
    except:
        raise NotImplementedError(f"Il tipo non è supportato")
    else:
        return result

Se il try va bene è ok, sennò da l'errore. Tutto dipende dai tipi che mettiamo nelle istanze. ATTENZIONE in questo caso ho definito il numero degli argomenti, ovvero 2. Se questo non viene rispettato si avrà un errore esterno. Potevo tenere *args e **kwargs per poi sommare con for x in args: result+=x ad esempio. Ho fatto cosi perchè si può, a scopo illustrativo

Adesso andiamo ad implementare la parte successiva, ovvero tutte le varianti della funzione con i diversi set di parametri che ci servono. RICORDA <U>solo il primo argomento fa da dispatcher.</U> Per prima cosa richiamo il decoratore @funzione.register. Poi definisco la funzione con gli argomenti che ci interessano. Da notare che il nome puo essere omesso per evitare ridondanza, ma è meno leggibile. Basta dargli un nome qualsiasi, deve avere senso per la leggibilità. È il decoratore @funzione.register che fa fede alla funzione originale

In [17]:
@funzione.register
def _(arg1: int, arg2):
    # qui possiamo metterci quello che vogliamo
    print("funzione con primo argomento un intero")

Ovviamente possiamo implementare la funzione come vogliamo. Ora possiamo ad esempio definire gli argomenti con due stringe

In [18]:
@funzione.register
def funzione_str(arg1: str, arg2):
    # qui possiamo metterci quello che vogliamo
    print("funzione con primo argomento una stringa")

Qui l'abbiamo chiamata funzione_str solo per la leggibilità, poteva chiamarsi sempre _.
Adesso andiamo a richiamare le funzioni

In [19]:
funzione(2, 2)
funzione(2, "banana")
print("   ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
funzione("5", (x for x in range(10)))
funzione("uno", "due")

funzione con primo argomento un intero
funzione con primo argomento un intero
   ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
funzione con primo argomento una stringa
funzione con primo argomento una stringa


Se richiamo una funzione che non abbiamo implementato con altri argomenti, si passa al fallback. Se l'implementazione col try funziona allota ritorna col return, altrimenti ho l'errore che ho scritto nel raise

In [20]:
funzione([1, 1],[2,2])

[1, 1, 2, 2]

Questo funziona, sono due liste, percio se le sommo si concatenano

In [21]:
try:
    funzione(None, 1)
except Exception as error:
    print("An exception occurred:", type(error).__name__, ":", error)

An exception occurred: NotImplementedError : Il tipo non è supportato


In questo caso ci dice che non è implementata quella funzione con l'argomento None, perchè sommare un None con qualsiasi tipo da errore al try. Infatti appare il messaggio che volevamo

Se interessati esiste una libreria che non è builtin che permette il dispatch con più argomenti. Si chiama <strong>multipledispatch</strong>. Consultare <a href="https://martinheinz.dev/blog/50"> questa pagina</a> per saperne di più

In certi casi è meglio usare la funzione di functools [partial](#partial)

Per maggiori dettagli la <a href="https://docs.python.org/3/library/functools.html">documentazione ufficiale di functools</a>.

# Classi


## Metodi Built-in

<a id='partialmethod'></a>
### partialmethod
Come in [partial](#partial) ma all'interno delle classi. Vediamo un esempio meno generico rispetto a quello visto in partial

In [22]:
from functools import partialmethod

In [23]:
class Stato:
    def __init__(self):
        self._stato = False
        
    @property
    def stato(self): #getter
        return self._stato
    def set_stato(self, nuovo): #setter
        self._stato = bool(nuovo)
        
    set_online = partialmethod(set_stato, True) #setter parzializzati
    set_offline = partialmethod(set_stato, False)    

In questo caso sfruttiamo partial per creare una funzione setter diversa per l'online e l'offline

In [24]:
s = Stato()
print(s.stato) # getter

s.set_online() # setter parzializzato 1
print(s.stato)

s.set_offline() # setter parzializzato 2
print(s.stato)

False
True
False


In questo esempio abbiamo visto come parzializzanado una funzione anche con un solo argomento, possiamo rendere il codice più leggibile.

Se invece dobbiamo cambiare l'implementazione del metodo in funzione degli argomenti che gli diamo, allora usare [@singledispatchmethod](#singledispatchmethod)

Per maggiori dettagli la <a href="https://docs.python.org/3/library/functools.html">documentazione ufficiale di functools</a>.

## Decoratori di metodi
### Creazione di un decoratore di metodi
Come visto nei decoratori di funzioni, possiamo creare anche i decoratori per i metodi. Il processo è sempre lo stesso, usando [@wraps](#wraps) della libreria functools

In [25]:
from functools import wraps

class Prova:
    @staticmethod
    def decoratore(metodo):
        @wraps(metodo)
        def wrap(self, *args, **kwargs):
            self.pippo = (1, 3, 4, 6, 8, 9, 11)
            self.numero = 0
            return metodo(self, *args, **kwargs)
        return wrap

<a id='staticmethod'></a>
### @staticmethod

<a id='property'></a>
### @property

È un decoratore per rendere pythoniche le funzioni getter setter e deleter

Nelle classi si possono creare attributi privati, che servono solo per uso interno diciamo,
a volte però è necessario settarli dall'esterno, perciò servono delle funzioni nella classe per,
settarle, ottenerne il valore, oppure cancellarle, ovvero funzioni dette getter, setter e deleter.
Il decoratore property lo fa in modo piu pythonico quindi piu leggibile e mantenibile.

In [26]:
class Teleconduttore:
    def __init__(self, p):
        self._pippo = p         # quando un attributo inizia con _, allora è privato

    @property                   # questo è il getter, ovvero una funzione che restituisce il valore
    def baudo(self):
        return self._pippo  

    @baudo.setter               # questo è il setter
    def baudo(self, val):
        self._pippo = val

    @baudo.deleter              # questo è il deleter
    def baudo(self):
        del self._pippo

Adesso lo andiamo a provare

In [27]:
sanremo = Teleconduttore(10)

print(sanremo.baudo)  # qui eseguo il getter per printarlo

sanremo.baudo = 5     # qui eseguo il setter e printo con il getter
print(sanremo.baudo)

del sanremo.baudo     # questo è il deleter che fa sparire _pippo dalla faccia della terra

10
5


Se adesso andiamo a printare baudo ci da errore

In [28]:
try:
    print(sanremo.baudo)
except Exception as error:
    print("An exception occurred:", type(error).__name__, ":", error)

An exception occurred: AttributeError : 'Teleconduttore' object has no attribute '_pippo'


Per maggiori dettagli la <a href="https://docs.python.org/3/library/functions.html">documentazione ufficiale delle built-in</a> di python.

<a id='singledispatchmethod'></a>
### @singledispatchmethod
#### Overloading di metodi
Come in [@singledispatch](#singledispatch) il dispatcher <U>è il type del primo argomento</U>, in questo caso pero <U>dopo il self o il cls</U> (si perchè puo essere usato anche in un <code class="inline"><em>@classmethod</em></code>).

Andiamo a richiamare la libreria, creiamo una classe e il metodo da decorare che funge da fallback, con i suoi dispatch

In [29]:
from functools import singledispatchmethod

In [30]:
class Negatore:
    @singledispatchmethod
    def neg(self, arg):
        raise NotImplementedError("Impossibile da negare")

    @neg.register
    def _(self, arg: int):
        return -arg

    @neg.register
    def _(self, arg: bool):
        return not arg

In [31]:
pippo = Negatore()
print(pippo.neg(5))

-5


Notare che il primo argomento deve essere sempre il self. Possiamo però usarla anche con le <code class="inline"><em>@classmethod</em></code>, ma anche le <code class="inline"><em>@staticmethod</em></code> e le <code class="inline"><em>@abstractmethod</em></code>. Per fare ciò dobbiamo annidare (nesting) i due decoratori, facendo attenzione all'ordine in cui lo facciamo:

In [32]:
class Negatore:
    @singledispatchmethod
    @classmethod
    def neg(cls, arg):
        raise NotImplementedError("Cannot negate a")

    @neg.register
    @classmethod
    def _(cls, arg: int):
        return -arg

    @neg.register
    @classmethod
    def _(cls, arg: bool):
        return not arg

In [33]:
print(Negatore.neg(5))

-5


Quindi prima il decoratore del singledispatch, poi quello del metodo, come il classmethod in questo caso. Ovviamente adesso invece del self dobbiamo mettere cls.

#### Caso particolare: Overloading nell' Overriding dei dunder methods

<p>In questo caso andiamo a lavorare sul dunder <code class="inline">__add__</code> in una classe</p>

In [34]:
from dataclasses import dataclass
from functools import singledispatchmethod

Una classe di tipo data che prende una nota in forma di stringa, inoltre ne ottiene il suo corrispettivo valore valore numerico (da noi scelto) enumerando una tupla a partire da 1

Quello che si vuole ottenere è che quando andiamo a sommare due oggetti della stessa classe, oppure un oggetto della classe con un intero, dia il valore numerico sommato. Se è maggiore di 12 si ritorna da capo a 1

In [35]:
@dataclass
class Nota:
    let: str

    def __post_init__(self):
        diesis = ('A', 'A#', 'B', 'C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#')
        bemolle = ('A', 'Bb', 'B', 'C', 'Db', 'D', 'Eb', 'E', 'F', 'Gb', 'G', 'Ab')
        pippo = diesis if self.let in diesis else bemolle
        for n, l in enumerate(pippo, 1):
            if l == self.let:
                self.num = n
    
    @singledispatchmethod
    def __add__(self, other):
        try:
            sum = self.num + other.num
        except:
            raise NotImplementedError(f"Il tipo non è supportato")
        else:
            sum = result - 12 if sum > 12 else sum
            return sum

    @__add__.register
    def _(self, n: int):
        sum = self.num + n
        sum = sum - 12 if sum > 12 else sum
        return sum

<p>Da notare che nella funzione base (quella di fallback) come argomenti abbiamo messo quelle di default del dunder <code class="inline">__add__</code>, ovvero self e other. Nel dispatcher abbiamo messo il self e poi l'argomento che dispatcha.</p>

Per maggiori dettagli <a href="https://docs.python.org/3/library/functools.html">la documentazione ufficiale di functools</a>.

<a id='dataclass'></a>
### @dataclass

Un decoratore studiato per impostare rapidamente una classe dedita ad ospitare dati

Questo decoratore permette di generare una classe con i dunder methods __init__, __repr__ e __eq__ gia preimpostati. L'__init__ infatti è completamente omesso, per questioni di leggibilità, ma gli attributi definiti nella classe sono li. Intanto importiamo dataclass da dataclasses

In [36]:
from dataclasses import dataclass

Ecco gli attributi che in realtà sono implemetati automaticamente nell'__init__. Attenzione, da notare che questi attributi, non possono essere definiti internamente alla classe. Non è nemmeno necessario esplicitare gli argomenti della clcasse

In [37]:
@dataclass
class Scala:
    tonica: int
    modo: int
    alterazione: str = "#"
    tipo: str = "Diatonica"

Tuttavia se è necessario definire alcuni attributi nella classe possiamo usare il dunder __post_init__, come se fosse l'__init__ classico delle classi, perdendo la bella leggibilità di prima purtroppo, con tutti i self

In [38]:
@dataclass
class Scala:
    tonica: int
    modo: int
    alterazione: str = "#"
    tipo: str = "Diatonica"
    
    def __post_init__(self):
        if self.tipo == "Diatonica" and self.modo == 1:
            self.intervalli = (2, 2, 1, 2, 2, 2, 1)

Adesso usciamo dalla classe e andiamo a creare gli oggetti per vedere altre specialità

In [39]:
scala1 = Scala(4, 3, "b")
print(scala1)

Scala(tonica=4, modo=3, alterazione='b', tipo='Diatonica')


La scala è stata creata ed è anche possibile printarla senza dichiarare l'istanza. Questo è possibile grazie al dunder __repr__ che viene creato automaticamente dal decoratore, in forma standard. Ora andiamo a provare il dunder __eq__

Creo un'altra scala e provo a valutare se è uguale a quella di prima

In [40]:
scala2 = Scala(4, 3, "#")
print(scala1 == scala2)

False


Ci dice ovviamente che è falso. Questo è possibile solo grazie al dunder __eq__

Se non ci piaciono come sono preimpostati i dunder, possiamo fare l'overriding

Andiamo a modificare il modo di rappresentazione. Basta ridefinire il dunder __repr__

In [41]:
@dataclass
class Scala:
    tonica: int
    modo: int
    alterazione: str = "#"
    tipo: str = "Diatonica"

    def __repr__(self):                                 
        return f"{self.tonica} - {self.modo} - {self.tipo}"

In [42]:
print(Scala(1,1))

1 - 1 - Diatonica


Abbiamo modificato il metodo di rappresentazione dell'oggetto. Possiamo modificare anche gli altri, ovvero __eq__, quindi come va a confrontare le classi, con quali attributi e in che modo, oppure volendo anche l'__init__, ma a quel punto non avrebbe proprio senso... Piuttosto usare il __post_init__ se necessario.

Se andiamo a rendere vero il flag order negli argomenti della classe, possiamo usare gli altri dunder di contronto __lt__, __le__, __gt__, __ge__, ovvero rispettivamente <, <=, > e >=.

ATTENZIONE</strong> questi dunder non possono essere overridati una volta attivato il Flag order. Se li devi modificare allora li devi esplicitare e tenere il flag false

In [43]:
@dataclass(order=True)
class Scala:
    tonica: int
    modo: int
    alterazione: str = "#"
    tipo: str = "Diatonica"

print(Scala(1, 1) < Scala(1, 2))
print(Scala(1, 1) >= Scala(1, 2))

True
False


Con il flag frozen=True non possiamo andare a ridefinire gli attributi nelle istanze

In [44]:
@dataclass(frozen=True)
class Scala:
    tonica: int
    modo: int
    alterazione: str = "#"
    tipo: str = "Diatonica"

scala = Scala(5, 5, "#")
try:
    scala.alterazione = "b"
except Exception as error:
    print("An exception occurred:", type(error).__name__, "–", error)

An exception occurred: FrozenInstanceError – cannot assign to field 'alterazione'


#### astuple, asdict

Oltre a dataclass, possiamo aggiungere altre cose carine

In [45]:
from dataclasses import dataclass, astuple, asdict

In [46]:
@dataclass
class Scala:
    tonica: int
    modo: int
    alterazione: str = "#"
    tipo: str = "Diatonica"

print((Scala(1, 1)))
print(astuple(Scala(1, 1)))         # prova, è meraviglioso
print(asdict(Scala(1, 1)))

Scala(tonica=1, modo=1, alterazione='#', tipo='Diatonica')
(1, 1, '#', 'Diatonica')
{'tonica': 1, 'modo': 1, 'alterazione': '#', 'tipo': 'Diatonica'}


Questo è ottimo per usare i dati al volo in forma di tupla o dizionario

Per ulteriori informazioni consultare la <a href="https://docs.python.org/3/library/dataclasses.html">documentazione ufficiale di dataclasses</a> di python

## Composizione (has-a)
### Un oggetto di una classe ha dentro l'oggetto di un'altra classe
Creo una classe per definire un punto, con al suo interno un metodo per definirne la distanza dal centro (0,0)

In [47]:
import math

class Punto:
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y

    def distanza(self):
        """Restituisce la distanza dall'origine"""
        return math.sqrt(self.x**2 + self.y**2)

    def __str__(self):
        return str((self.x, self.y))

In [48]:
p = Punto()
print(p)
print(p.distanza())

(0, 0)
0.0


In [49]:
p = Punto(3,4)
print(p)
print(p.distanza())

(3, 4)
5.0


Adesso creo un'atra classe che definisce il cerchio, usando l'oggetto della classe punto come suo centro sul piano cartesiano. Anche qui creo un metodo che fornisce la distanza dal centro del piano (0,0).

In [50]:
class Cerchio:
    def __init__(self, raggio=1, centro=Punto()):
        self.raggio = raggio
        self.centro = centro

    def area(self):
        return math.pi * self.raggio**2

    def distanza(self):
        """Di nuovo, restituisce la distanza dall'origine, chiamando il metodo di Punto"""
        return self.centro.distanza()

    def __str__(self):
        return "raggio: %f, centro: %s" %(self.raggio, self.centro)

In [51]:
c = Cerchio()
print(c)

raggio: 1.000000, centro: (0, 0)


In [52]:
c = Cerchio(10, Punto(3,4))
print(c)
print(c.distanza())
print(c.area())

raggio: 10.000000, centro: (3, 4)
5.0
314.1592653589793


## Ereditarietà (is-a)
### Quando un oggetto di una classe è un oggetto di un'altra classe
Può capitare in programmazione ad oggetti che dobbiamo descrivere oggetti diversi, ma molto simili fra loro da un punto di vista proprio di attributi e metodi. Il classico esempio è gli oggetti persona e impiegato. Infatti, l'impiegato È A TUTTI GLI EFFETTI una persona, perciò ne condivide le caratteristiche, più altre caratteristiche in piu che lo distingue dalla semplice persona, o da un dirigente ad esempio. Siccome l'esempio persona con nome, cognome, email e codice fiscale mi sta sul cazzo, faccio un esempio con gli animali.

Andro a creare tre classi concatenate, ovvero un canide che è un mammifero che è un animale. Ciascun metodo relativo alla classificazione lo nomino in modo diverso, perchè alla fine dimostro che posso usarli distintamente senza fatica.

In [53]:
class Animale:
    def __init__(self, nome):
        self.nome = nome
    def __str__(self):
        return "Nome: " + self.nome 
    def classificazione_base(self):
        dominio = "eukaryota"
        regno = "animalia"
        return dominio + "\n" + regno + "\n"

In [54]:
class Mammifero(Animale):
    def __init__(self, nome, dieta):
        self.dieta = dieta
        Animale.__init__(self, nome)
    def __str__(self):
        return Animale.__str__(self) + " - " + "Dieta: " + self.dieta
    def classificazione_intermedia(self):
        phylum = "chordata"
        subphylum = "vertebrata"
        classe = "mammalia"
        return Animale.classificazione_base(self) + phylum + "\n" + subphylum + "\n" + classe + "\n"

In [55]:
class Canide(Mammifero):
    def __init__(self, nome, dieta, habitat):
        self.habitat = habitat
        Mammifero.__init__(self, nome, dieta)
    def __str__(self):
        return Mammifero.__str__(self) + " - " + "Habitat: " + self.habitat
    def classificazione_completa(self):
        ordine = "carnivora"
        famiglia = "canidae"
        pippo = self.nome.split()
        genere = pippo[0]
        specie = pippo[1]
        return Mammifero.classificazione_intermedia(self) + ordine + "\n" + famiglia + "\n" + genere + "\n" + specie + "\n"

In [56]:
cane = Canide("canis lupus", "onnivoro", "urbano")
print(cane) # esprime il dunder method __str__
print("\n")
print(cane.classificazione_completa())

Nome: canis lupus - Dieta: onnivoro - Habitat: urbano


eukaryota
animalia
chordata
vertebrata
mammalia
carnivora
canidae
canis
lupus



<p>Sia il metodo dunder <code class="inline">__str__</code> che i metodi di classificazione concatenati funzionano a dovere se richiamato dalla classe ultima derivata. In ogni caso posso andare a richiamare anche il singolo metodo della classe base o di quella intermedia</p>

In [57]:
print(cane.classificazione_base())

eukaryota
animalia



Lo stesso vale per gli attributi, infatti qui chiamo il singolo attributo dieta nella classe intermedia

In [58]:
print(cane.dieta)

onnivoro


Tutto bello, anche se questo non è proprio il modo più bello per definire l'ereditarietà in Python. Finchè c'è un'ereditarietà lineare come in questo caso, il discorso non fa una piega. Le cose si complicano quando vi sono eredità multiple e concatenate.
Per questo motivo è stato introdotto <code class="inline">super()</code>.

### Eredità singole o a catena: la funzione super()
Rimaniamo nell'esempio di prima e introduciamo la funzione <code class="inline">super()</code>:

In [59]:
class Animale:
    def __init__(self, nome):
        self.nome = nome
    def __str__(self):
        return "Nome: " + self.nome 
    def classificazione_base(self):
        dominio = "eukaryota"
        regno = "animalia"
        return dominio + "\n" + regno + "\n"

class Mammifero(Animale):
    def __init__(self, nome, dieta):
        self.dieta = dieta
        super().__init__(nome)
    def __str__(self):
        return super().__str__() + " - " + "Dieta: " + self.dieta
    def classificazione_intermedia(self):
        phylum = "chordata"
        subphylum = "vertebrata"
        classe = "mammalia"
        return super().classificazione_base() + phylum + "\n" + subphylum + "\n" + classe + "\n"

Nella classe madre è tutto come prima, nella intermedia invece è apparso <code class="inline">super().__init__(nome)</code>. <U>La funzione <code class="inline">super()</code> ritorna un oggetto che rappresenta la classe parent</U>. In questo modo sto passando alla classe parent la variabile name da cui dovrò attingere per avere una classificazione completa, esattamente come ho fatto prima.

<p>Inoltre nel dunder method <code class="inline">__str__</code> anche abbiamo <code class="inline">super()</code> che recupera quello che è stato definito gia nella classe madre e aggiunge ulteriori informazioni</p>

<p>Anche nel metodo <code class="inline">classificazione_intermedia(self)</code> andiamo a fetchare quello che è stato gia definito in <code class="inline">super().classificazione_base()</code>, per farlo concatenare con ulteriori informazioni.</p>

Andando a guardare l'ultima classe ereditaria:

In [60]:
class Canide(Mammifero):
    def __init__(self, nome, dieta, habitat):
        self.habitat = habitat
        super().__init__(nome, dieta)
    def __str__(self):
        return super().__str__() + " - " + "Habitat: " + self.habitat
    def classificazione_completa(self):
        ordine = "carnivora"
        famiglia = "canidae"
        pippo = self.nome.split()
        genere = pippo[0]
        specie = pippo[1]
        return super().classificazione_intermedia() + ordine + "\n" + famiglia + "\n" + genere + "\n" + specie + "\n"

Ci siamo capiti insomma. Per concludere gli output:

In [61]:
cane = Canide("canis lupus", "onnivoro", "urbano")
print(cane)
print("\n")
print(cane.classificazione_completa())

Nome: canis lupus - Dieta: onnivoro - Habitat: urbano


eukaryota
animalia
chordata
vertebrata
mammalia
carnivora
canidae
canis
lupus



NOTARE BENE: Andiamo a paragonare queste due espressioni complementari nei due esempi, il primo senza super e il secondo con:
<p>
    <code class="inline">Animale.__init__(self, nome)</code>
    <br>
    <code class="inline">super().__init__(nome)</code>
</p>
Nella funzione super <U>non si usa mai il parametro self, nemmeno in presenza di metodi</U> perchè insieme alla classe base, è gia implicito, con una dichiarazione dei parametri un po insolita:
<p><code class="inline">super(classe_sestessa, self)</code></p>
L'eredità singola può essere definita lineare o a catena, perchè sono collegate una dopo l'altra in successione. In senso gerarchico, l'esecuzione del codice viene impartita in senso verticale.

     Animale        ░▒[>>> Cane.__mro__
        ↑           ░▒(__main__.Canide, __main__.Mammifero, __main__.Animale, object)
    Mammifero       ░▒
        ↑           ░▒                         Animale
      Cane          ░▒                           |
                    ░▒ Astrazione   ★☆☆        Mammifero(Animale)     eredità singola a catena
     super()        ░▒ Ambiguità    ☆☆☆          |
                    ░▒ Complessità  ★☆☆        Cane(Mammifero)
<p>
Per conoscere l'eredità di una specifica classe, si usa l'attributo dunder <code class="inline">__mro__</code>. Tra un po scopriremo che questo attributo, oltre ad essere informativo per il programmatore, è anche legato al funzionamento di <code class="inline">super()</code></p>

In [62]:
Canide.__mro__

(__main__.Canide, __main__.Mammifero, __main__.Animale, object)

<p>Ma a cosa serve questo <code class="inline">super()</code>? È solo accanimento pythonico? E soprattutto, come fa a capire in quale classe va a finire? Ce lo dice appunto l'attributo dunder <code class="inline">__mro__</code> che spieghiamo nel prossimo capitolo:</p>

### Eredità multipla: super() e l'attributo dunder \_\_mro__
Vediamo come <code class="inline">super()</code> gestisce l'ereditarietà multipla e come fa a capire dove deve andare. 
Creo tre classi A1, A2, A3 con dei metodi con lo stesso nome, ma con output diversi e creo una classe B con eredità multipla:

In [63]:
class A1:
    def pippo(self):
        print("pippo_A1")
class A2:
    def pippo(self):
        print("pippo_A2")
class A3:
    def pippo(self):
        print("pippo_A3")

class B(A2, A1, A3):  # NOTARE ORDINE DI INSERMENTO
    def pippo(self):
        pass

<p>L'ereditarietà viene suddivisa in gerarchia secondo l'ordine di inserimento nella classe derivata. Per capire meglio usiamo l'attributo dunder <code class="inline">__mro__</code></p>

In [64]:
B.__mro__

(__main__.B, __main__.A2, __main__.A1, __main__.A3, object)

Alla fine, ovvero all'apice della gerarchia c'è la classe object che chiude la catena. 

<p>La funzione <code class="inline">super()</code> chiama l'attributo dunder <code class="inline">__mro__</code> che a sua volta <U>legge l'ordine di ereditarietà che abbiamo espresso nell'argomento della classe B</U>. 
    <br>
    <br>
    In un certo senso, B è child prima di A2, poi di A1 e infine di A3 in questo ordine. Il nostro <code class="inline">__mro__</code> si assicura che questo ordine venga rispettato al cospetto di <code class="inline">super()</code>.

Andiamo ad inserire <code class="inline">super()</code> nelle classi di interesse</p>

In [65]:
class A1:
    def pippo(self):
        print("pippo_A1")
        super().pippo()
class A2:
    def pippo(self):
        print("pippo_A2")
        super().pippo()
class A3:
    def pippo(self):
        print("pippo_A3")
        # NOTARE CHE QUI NON C'È SUPER(), DAREBBE ERRORE
        
class B(A2, A1, A3):  # NOTARE ORDINE DI INSERMENTO
    def pippo(self):
        super().pippo()

Se vado a richiamare l'oggetto e il metodo vediamo cosa ottengo

In [66]:
b = B()
b.pippo()

pippo_A2
pippo_A1
pippo_A3


<p>Viene appunto eseguito il codice nell'ordine fornito da <code class="inline">__mro__</code>.
    <br>
    <br>
    L'esecuzione del codice viene impartita in senso verticale tra la classe child e il primo parent, poi in senso orizzontale tra i parent successivi
</p>

    A2  →  A1  →  A3    ░▒[>>> B.__mro__
       ↖                ░▒(__main__.B, __main__.A2, __main__.A1, __main__.A3, object)
         ↖              ░▒
           B            ░▒                             
                        ░▒ Astrazione   ★☆☆         A2  A1 A3       eredità multipla
        super()         ░▒ Ambiguità    ★☆☆           \ | /
                        ░▒ Complessità  ★☆☆             B(A2, A1, A3)

### Eredità combinata o a diamante: introduzione di super() [Python 2.2] e \_\_mro__ [Python 2.3]
<p>Questo esempio chiarisce perchè sono stati introdotti questi strumenti perchè questo caso genera un problema di ridondanza ereditaria, che non era facilmente gestibile.
    <br>
    <br>
    Impostiamo il caso di cosiddetta eredità a diamante senza <code class="inline">super()</code> e senza <code class="inline">__mro__</code>
    <br>
    <br>
    NOTA BENE: nel prossimo blocco di codice che ho scritto per ricreare l'inconsistenza di questo specifico caso, non avrebbe assolutamente senso consultare <code class="inline">__mro__</code>, perchè la discendenza ereditaria estrapolata dall'ordine in cui vengono espresse le eredità stesse, è coerente solo ed esclusivamente con l'utilizzo di <code class="inline">super()</code> che qui è volutamente omesso.</p>

In [67]:
class A:
    def __init__(self):
        print("arrivo in A\n⇓ ⇓ ⇓ ⇓ ⇓ ⇓")
        print("in A eseguo __init__()")
        # codice da eseguire in A__init__()

class B1(A):
    def __init__(self):
        print("in B1 chiamo A")
        A.__init__(self)
        print("in B1 eseguo __init__()")
        # codice da eseguire in B1__init__()

class B2(A):
    def __init__(self):
        print("in B2 chiamo A")
        A.__init__(self)
        print("in B2 eseguo __init__()")
        # codice da eseguire in B2__init__()

class C(B1, B2):
    def __init__(self):
        print("in C chiamo B1")
        B1.__init__(self)
        print("in C chiamo B1")
        B2.__init__(self)
        print("in C eseguo __init__()")
        # codice da eseguire in C__init__()

c = C()

in C chiamo B1
in B1 chiamo A
arrivo in A
⇓ ⇓ ⇓ ⇓ ⇓ ⇓
in A eseguo __init__()
in B1 eseguo __init__()
in C chiamo B1
in B2 chiamo A
arrivo in A
⇓ ⇓ ⇓ ⇓ ⇓ ⇓
in A eseguo __init__()
in B2 eseguo __init__()
in C eseguo __init__()


<p>La classe B deve leggere gli <code class="inline">__init__()</code> delle sue due classi parent A1 e A2, che a loro volta hanno come classe parent A. In questo caso le due classi B1 e B2 vengono dette classi siblings, perchè hanno entrambe la stessa classe parent A e sono combinate da C. 
    <br>
    <br>
    Ed ecco la ridondanza ereditaria, un bel casino logico che inoltre <U>è molto poco pythonico se rifletti attentamente.</U> <br>
    Ci si sono messi di impegno e hanno creato questi due strumenti che vanno a risolvere:</p>

In [68]:
class A:
    def __init__(self):
        print("arrivo in A\n⇓ ⇓ ⇓ ⇓ ⇓ ⇓")
        print("in A eseguo __init__()")
        # codice da eseguire in A__init__()

class B1(A):
    def __init__(self):
        print("in B1 eseguo super()")
        super().__init__()
        print("in B1 eseguo __init__()")
        # codice da eseguire in B1__init__()

class B2(A):
    def __init__(self):
        print("in B2 eseguo super()")
        super().__init__()
        print("in B2 eseguo __init__()")
        # codice da eseguire in B2__init__()

class C(B1, B2):
    def __init__(self):
        print("in C eseguo super()")
        super().__init__()
        print("in C eseguo __init__()")
        # codice da eseguire in C.__init__()

c = C()

in C eseguo super()
in B1 eseguo super()
in B2 eseguo super()
arrivo in A
⇓ ⇓ ⇓ ⇓ ⇓ ⇓
in A eseguo __init__()
in B2 eseguo __init__()
in B1 eseguo __init__()
in C eseguo __init__()


In [69]:
C.__mro__

(__main__.C, __main__.B1, __main__.B2, __main__.A, object)

<p>
    Cosa è successo? Parte C, la prima istruzione è super, perciò legge nell'<code class="inline">__mro__</code> qual'è il successivo, ovvero B1. Qui accade la stessa cosa perchè c'è il super come prima istruzione, e quindi va avanti in B2. Di nuovo la stessa cosa e va ad A. Dopodichè printa "eseguo __init__ in A", torna indietro ad A2 e printa e cosi via.
    <br>
    <br>
    Nel seguente grafico ho inserito anche l'esecuzione di <code class="inline">__init__()</code> per restare coerente con l'output del codice di sopra, così da rappresentare un'eventuale esecuzione di codice presente nelle inizializzazioni delle classi dopo l'escalation dei super()
</p>

        A  ⇒ ⇒ ⇒ ⇒ ⇒ ⇒ ⇒ ⇒ ⇒  A         ░▒[>>> C.__mro__
          ↖                     ↘       ░▒(__main__.C, __main__.B1, __main__.B2, __main__.A, object)
    B1  →  B2             B1  ←  B2     ░▒
      ↖                     ↘           ░▒                               A
        C                     C         ░▒                            ┌ ─┴─ ┐
                                        ░▒ Astrazione   ★★☆          B1(A)  B2(A)       eredità a diamante
     super()              __init__()    ░▒ Ambiguità    ★★☆            \   /
                                        ░▒ Complessità  ★★☆              C(B1, B2)
<p>Questo caso combina un caso di eredità multipla, con un doppio caso di eredità singola verso lo stesso parent. Per questo motivo, le due classi centrali B1 e B2 sono dette siblings di A. Infatti siblings è un termine complementare di child, contestualizzato al caso in cui due classi hanno lo stesso insieme di parents. Per essere chiari, in questo caso B1 e B2 sono siblings attraverso eredità singola, ma possono anche essere siblings se avessero avuto la stessa identica eredità multipla, a condizione che entrambe abbiano gli stessi parents multipli e nello stesso ordine. In altre parole se B1(A1, A2) mentre B2(A2), allora non sono siblings e nemmeno B1(A1, A2) e B2(A2, A1) sono considerabili siblings, essendo l'ordine di eredità invertito.
    <br>
    <br>
    Se ci pensi è semplice, è l'<code class="inline">__mro__</code> che parla chiaro, e se lo facessi sia a B1 che a B2, gli output devono essere identici per essere considerati siblings!</p>

### Casi notevoli di Eredità
A seguire una serie di casi particolari degni di nota

    ░▒[>>> E.__mro__
    ░▒(__main__.E, __main__.D1, __main__.D2, __main__.C, __main__.B1, __main__.B2, __main__.A, object)
    ░▒
    ░▒                                A
    ░▒                             ┌ ─┴─ ┐
    ░▒                            B1(A)  B2(A)
    ░▒                              \   /
    ░▒                                C(B1, B2)         eredità a mosaico
    ░▒                             ┌ ─┴─ ┐
    ░▒ Astrazione   ★★☆           D1(C)  D2(C)
    ░▒ Ambiguità    ★★☆             \   /
    ░▒ Complessità  ★★★               E(D1, D2)

<p>Estremamente flessibile e potente, ma può diventare molto complessa e difficile da gestire. Richiede una chiara comprensione della <code class="inline">__mro__</code> e delle interazioni tra classi. Per semplicità di esempio questo è chiaramente un doppio diamante, ma non deve essere per forza simmetrico. Le successioni di eredità multiple e singole possono essere sfalzate a volontà elevandone ancora di più la complessità e la potenza.</p>

    ░▒[>>> B.__mro__
    ░▒TypeError: Cannot create a consistent method resolution order (MRO) for bases A, B
    ░▒
    ░▒                               A(B)
    ░▒                            ┌ ─┴─ ┐
    ░▒ Astrazione   ★★★           │     │              eredità circolare (non attuabile in python)
    ░▒ Ambiguità    ★★★           └ ─┬─ ┘
    ░▒ Complessità  ☆☆☆              B(A)

In realtà ne ricevi un'altro prima di errore, relativo proprio alla definizioni delle classi. Se dall'alto verso il basso del codice prima definisco A(B) e poi B(A) ottengo l'errore: NameError: name 'B' is not defined.

L'ereditarietà circolare non ha senso in pratica perché viola i principi di progettazione orientata agli oggetti. Un tale design porterebbe a una confusione logica e ambiguità su quale classe dovrebbe ereditare quale metodo o attributo.

### Richiamo di metodi, attributi e attributi privati con super()

### Eredità con Sovrascrittura di Metodi: l'Overriding

In [70]:
class Animal:
    def speak(self):
        return "Some generic sound"

class Dog(Animal):
    def speak(self):
        return "Woof!"  # Questo è l'overriding del metodo 'speak'

# Utilizzo delle classi
generic_animal = Animal()
dog = Dog()

print(generic_animal.speak())  # Output: "Some generic sound"
print(dog.speak())             # Output: "Woof!"


Some generic sound
Woof!


## Ereditarietà astratta

In [71]:
from abc import ABC, abstractmethod

class Shape(ABC):
    @abstractmethod
    def area(self):
        pass

class Circle(Shape):
    def __init__(self, radius):
        self.radius = radius

    def area(self):
        return 3.14159 * (self.radius ** 2)

class Rectangle(Shape):
    def __init__(self, width, height):
        self.width = width
        self.height = height

    def area(self):
        return self.width * self.height

# Tentativo di istanziare la classe astratta darà errore
# shape = Shape()  # TypeError: Can't instantiate abstract class Shape with abstract methods area

# Utilizzo delle classi derivate
circle = Circle(5)
rectangle = Rectangle(4, 6)

print(circle.area())  # Output: 78.53975
print(rectangle.area())  # Output: 24

78.53975
24
